In [1]:
import pandas as pd
import numpy as np
import re
import datetime
from pandas.core.frame import DataFrame
import matplotlib.pyplot as plt
import math
import statistics
from statistics import mean

In [2]:
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('ggplot')
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline, make_pipeline
from scipy.stats import skew
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.isotonic import IsotonicRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import ElasticNet, SGDRegressor, BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from xgboost.sklearn import XGBRegressor

# Pre-processing

## Training set  (Pre-processed)

In [3]:
filename='./Data/df_train1_skew.csv'
df_train1 = pd.read_csv(filename)

In [4]:
df_train1

,playtime_forever,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,pur_rel_itv,log_price,log_total_positive_reviews,log_total_negative_reviews
0,0.000000,False,3700,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...",2018-07-02,2013-12-10,372.0,96.0,1665,8.216358,5.921578,4.574711
1,0.016667,True,0,RPG,"Single-player,Partial Controller Support","Mod,Utilities,RPG,Game Development,Singleplaye...",2016-11-26,2015-08-12,23.0,0.0,472,0.000000,3.178054,0.000000
2,0.000000,False,5000,"Adventure,Casual,Indie","Single-player,Full controller support,Steam Tr...","Point & Click,Adventure,Story Rich,Comedy,Indi...",2018-07-02,2014-01-28,3018.0,663.0,1616,8.517393,8.012681,6.498282
3,1.533333,False,9900,"Action,RPG","Single-player,Multi-player,Steam Achievements,...","Medieval,RPG,Open World,Strategy,Sandbox,Actio...",2016-11-28,2010-03-31,63078.0,1746.0,2434,9.200391,11.052143,7.465655
4,22.333333,False,4800,"Action,Indie,Strategy","Single-player,Co-op,Steam Achievements,Full co...","Tower Defense,Co-op,Action,Strategy,Online Co-...",2018-03-04,2012-07-30,8841.0,523.0,2043,8.476580,9.087268,6.261492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,0.783333,False,8800,"Action,Simulation","Single-player,Multi-player,Online Multi-Player...","Simulation,Action,VR,Flight",2017-11-24,2016-12-20,150.0,91.0,339,9.082621,5.017280,4.521789
336,0.000000,False,6800,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Batman,Action,Open World,Superhero,Stealth,Adv...",2018-08-15,2015-06-23,19008.0,4849.0,1149,8.824825,9.852668,8.486734
337,0.000000,False,8300,"Action,Adventure,RPG","Single-player,Steam Achievements,Full controll...","Action,Hack and Slash,Adventure,RPG,Open World...",2018-01-30,2015-11-05,5099.0,1719.0,817,9.024131,8.536996,7.450080
338,0.000000,False,6800,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Action,FPS,Adventure,Shooter,Cyberpunk,Robots,...",2017-09-23,2016-06-03,718.0,159.0,477,8.824825,6.577861,5.075174


### Read One-hot matrix from csv - process see 1118_genres&categories&tags.ipynb

In [5]:
filename='./Data/train_One_hot_genres.csv'
dfg_train = pd.read_csv(filename)

In [6]:
df_train2 = pd.concat([df_train1, dfg_train], axis=1, sort=False)
df_train2

,playtime_forever,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,...,Modeling,Nudity,RPG,Racing,SexualContent,Simulation,Sports,Strategy,Utilities,Violent
0,0.000000,False,3700,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...",2018-07-02,2013-12-10,372.0,96.0,...,0,0,0,0,0,0,0,0,0,0
1,0.016667,True,0,RPG,"Single-player,Partial Controller Support","Mod,Utilities,RPG,Game Development,Singleplaye...",2016-11-26,2015-08-12,23.0,0.0,...,0,0,1,0,0,0,0,0,0,0
2,0.000000,False,5000,"Adventure,Casual,Indie","Single-player,Full controller support,Steam Tr...","Point & Click,Adventure,Story Rich,Comedy,Indi...",2018-07-02,2014-01-28,3018.0,663.0,...,0,0,0,0,0,0,0,0,0,0
3,1.533333,False,9900,"Action,RPG","Single-player,Multi-player,Steam Achievements,...","Medieval,RPG,Open World,Strategy,Sandbox,Actio...",2016-11-28,2010-03-31,63078.0,1746.0,...,0,0,1,0,0,0,0,0,0,0
4,22.333333,False,4800,"Action,Indie,Strategy","Single-player,Co-op,Steam Achievements,Full co...","Tower Defense,Co-op,Action,Strategy,Online Co-...",2018-03-04,2012-07-30,8841.0,523.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,0.783333,False,8800,"Action,Simulation","Single-player,Multi-player,Online Multi-Player...","Simulation,Action,VR,Flight",2017-11-24,2016-12-20,150.0,91.0,...,0,0,0,0,0,1,0,0,0,0
336,0.000000,False,6800,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Batman,Action,Open World,Superhero,Stealth,Adv...",2018-08-15,2015-06-23,19008.0,4849.0,...,0,0,0,0,0,0,0,0,0,0
337,0.000000,False,8300,"Action,Adventure,RPG","Single-player,Steam Achievements,Full controll...","Action,Hack and Slash,Adventure,RPG,Open World...",2018-01-30,2015-11-05,5099.0,1719.0,...,0,0,1,0,0,0,0,0,0,0
338,0.000000,False,6800,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Action,FPS,Adventure,Shooter,Cyberpunk,Robots,...",2017-09-23,2016-06-03,718.0,159.0,...,0,0,0,0,0,0,0,0,0,0


## Testing set  (Pre-processed)

In [7]:
filename='./Data/df_test1_skew.csv'
df_test = pd.read_csv(filename)

In [9]:
df_test

,id,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,pur_rel_itv,log_price,log_total_positive_reviews,log_total_negative_reviews
0,0,False,3500,"Action,Adventure","Single-player,Full controller support","Action,Adventure,Horror,Third Person,Singlepla...",2018-10-28,2012-05-22,2607.0,1122.0,2350.0,8.160804,7.866339,7.023759
1,1,False,11600,"Action,Adventure,Strategy","Single-player,Multi-player,Online Multi-Player...","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...",2019-07-20,2018-04-24,5762.0,2235.0,452.0,9.358847,8.659213,7.712444
2,2,False,2100,"Indie,Simulation,Strategy","Single-player,Steam Achievements,Steam Trading...","Strategy,Simulation,Indie,Political,Cold War,P...",2019-07-16,2017-03-20,687.0,133.0,848.0,7.650169,6.533789,4.897840
3,3,False,3600,"Action,Strategy","Single-player,Multi-player,Co-op","Strategy,Action,Military,Tactical",2018-01-30,2007-07-17,67.0,39.0,3850.0,8.188967,4.219508,3.688879
4,4,False,3400,"Action,Adventure","Single-player,Co-op,Steam Achievements,Full co...","Open World,Action,Comedy,Co-op,Third-Person Sh...",2017-02-24,2013-08-22,40344.0,3708.0,1282.0,8.131825,10.605223,8.218518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,85,False,5000,Action,"Single-player,Steam Achievements,Full controll...","Western,FPS,Action,Story Rich,Bullet Time,Shoo...",2018-03-23,2013-05-22,9334.0,516.0,1766.0,8.517393,9.141526,6.248043
86,86,False,3600,"Action,Adventure",Single-player,"Adventure,Action,Ninja,Stealth,Cute,Singleplay...",2018-03-03,2009-09-14,1245.0,176.0,3092.0,8.188967,7.127694,5.176150
87,87,False,11200,"Simulation,Strategy","Single-player,Multi-player,Cross-Platform Mult...","Grand Strategy,Strategy,Historical,Simulation,...",2018-02-18,2013-08-13,33128.0,6350.0,1650.0,9.323758,10.408164,8.756368
88,88,True,0,"Action,Adventure,RPG,Simulation,Sports,Strategy","Single-player,Multi-player,Co-op,Online Co-op,...","VR,RPG,Action,Sports,Simulation,Adventure,Stra...",2017-11-19,2017-06-19,5.0,0.0,153.0,0.000000,1.791759,0.000000


### Read One-hot Matrix from csv - process see 1118_genres&categories&tags.ipynb

In [10]:
filename='./Data/test_One_hot_genres.csv'
dfg_test = pd.read_csv(filename)

In [11]:
df_test2 = pd.concat([df_test, dfg_test], axis=1, sort=False)
df_test2

,id,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,...,Modeling,Nudity,RPG,Racing,SexualContent,Simulation,Sports,Strategy,Utilities,Violent
0,0,False,3500,"Action,Adventure","Single-player,Full controller support","Action,Adventure,Horror,Third Person,Singlepla...",2018-10-28,2012-05-22,2607.0,1122.0,...,0,0,0,0,0,0,0,0,0,0
1,1,False,11600,"Action,Adventure,Strategy","Single-player,Multi-player,Online Multi-Player...","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...",2019-07-20,2018-04-24,5762.0,2235.0,...,0,0,0,0,0,0,0,1,0,0
2,2,False,2100,"Indie,Simulation,Strategy","Single-player,Steam Achievements,Steam Trading...","Strategy,Simulation,Indie,Political,Cold War,P...",2019-07-16,2017-03-20,687.0,133.0,...,0,0,0,0,0,1,0,1,0,0
3,3,False,3600,"Action,Strategy","Single-player,Multi-player,Co-op","Strategy,Action,Military,Tactical",2018-01-30,2007-07-17,67.0,39.0,...,0,0,0,0,0,0,0,1,0,0
4,4,False,3400,"Action,Adventure","Single-player,Co-op,Steam Achievements,Full co...","Open World,Action,Comedy,Co-op,Third-Person Sh...",2017-02-24,2013-08-22,40344.0,3708.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,85,False,5000,Action,"Single-player,Steam Achievements,Full controll...","Western,FPS,Action,Story Rich,Bullet Time,Shoo...",2018-03-23,2013-05-22,9334.0,516.0,...,0,0,0,0,0,0,0,0,0,0
86,86,False,3600,"Action,Adventure",Single-player,"Adventure,Action,Ninja,Stealth,Cute,Singleplay...",2018-03-03,2009-09-14,1245.0,176.0,...,0,0,0,0,0,0,0,0,0,0
87,87,False,11200,"Simulation,Strategy","Single-player,Multi-player,Cross-Platform Mult...","Grand Strategy,Strategy,Historical,Simulation,...",2018-02-18,2013-08-13,33128.0,6350.0,...,0,0,0,0,0,1,0,1,0,0
88,88,True,0,"Action,Adventure,RPG,Simulation,Sports,Strategy","Single-player,Multi-player,Co-op,Online Co-op,...","VR,RPG,Action,Sports,Simulation,Adventure,Stra...",2017-11-19,2017-06-19,5.0,0.0,...,0,0,1,0,0,1,1,1,0,0


# Feature selection

In [12]:
for d in zip(df_train2, df_test2):
    if d[0] != d[1]:
        print (d)

('playtime_forever', 'id')


In [13]:
scaler = StandardScaler()

X = df_train2.drop(columns=['playtime_forever','is_free','price','total_positive_reviews','total_negative_reviews','genres', 'categories', 'tags', 'purchase_date','release_date'])
test_X = df_test2.drop(columns=['id', 'is_free','price','total_positive_reviews','total_negative_reviews','genres','categories', 'tags', 'purchase_date','release_date'])
y= df_train2.playtime_forever

X_scaled = scaler.fit(X).transform(X)
y_log = np.log(df_train2.playtime_forever + 1)
test_X_scaled = scaler.transform(test_X)

## PCA 

In [14]:
components = X.shape[1]
pca = PCA(n_components = components)

In [15]:
X_scaled=pca.fit_transform(X_scaled)
test_X_scaled = pca.transform(test_X_scaled)

In [16]:
X_scaled.shape, test_X_scaled.shape

((340, 26), (90, 26))

# Modeling & Evaluation 

In [17]:
# define cross validation stragegy RMSE
def rmse_cv(model,X,y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=5))
    return rmse

In [18]:
models = [LinearRegression(),Ridge(),Lasso(),RandomForestRegressor(),GradientBoostingRegressor(),SVR(),LinearSVR(),
          ElasticNet(),SGDRegressor(),BayesianRidge(),KernelRidge(),
          ExtraTreesRegressor(),XGBRegressor(verbosity=0),AdaBoostRegressor()]

In [19]:
names = ["LR", "Ridge", "Lasso", "RF", "GBR", "SVR", "LinSVR", "Ela","SGD","Bay","Ker","Extra","Xgb","Ada"]
for name, model in zip(names, models):
    score = rmse_cv(model, X_scaled, y_log)
    print("{}: {:.6f}, {:.4f}".format(name,score.mean(),score.std()))

LR: 64634338419713.460938, 113890762246797.7500
Ridge: 0.819401, 0.0988
Lasso: 0.883973, 0.1248
RF: 0.867403, 0.1825
GBR: 0.865374, 0.1532
SVR: 0.809532, 0.1458
LinSVR: 0.920410, 0.1602
Ela: 0.883973, 0.1248
SGD: 1.261196, 0.8293
Bay: 0.809564, 0.1128
Ker: 6.066363, 4.6184
Extra: 0.865776, 0.1599
Xgb: 0.855547, 0.1448
Ada: 0.868026, 0.1364


## Fine Tuning Parameters

In [20]:
class grid():
    def __init__(self,model):
        self.model = model
    
    def grid_get(self,X,y,param_grid):
        grid_search = GridSearchCV(self.model,param_grid,cv=5, scoring="neg_mean_squared_error")
        grid_search.fit(X,y)
        print(grid_search.best_params_, np.sqrt(-grid_search.best_score_))
        grid_search.cv_results_['mean_test_score'] = np.sqrt(-grid_search.cv_results_['mean_test_score'])
        print(pd.DataFrame(grid_search.cv_results_)[['params','mean_test_score','std_test_score']])

In [24]:
# Ridge

grid(Ridge()).grid_get(X_scaled,y_log,{'alpha':[81,82,83,84,85,86,87,88,89,90]})

{'alpha': 84} 0.8151276889470095
          params  mean_test_score  std_test_score
0  {'alpha': 81}         0.815133        0.184473
1  {'alpha': 82}         0.815130        0.184648
2  {'alpha': 83}         0.815128        0.184823
3  {'alpha': 84}         0.815128        0.184996
4  {'alpha': 85}         0.815129        0.185168
5  {'alpha': 86}         0.815131        0.185338
6  {'alpha': 87}         0.815134        0.185507
7  {'alpha': 88}         0.815138        0.185676
8  {'alpha': 89}         0.815144        0.185843
9  {'alpha': 90}         0.815150        0.186008


In [25]:
# RF - USABLE

grid(RandomForestRegressor()).grid_get(X_scaled,y_log,{'max_features': ["auto","sqrt","log2"]})

{'max_features': 'log2'} 0.8244312565731198
                     params  mean_test_score  std_test_score
0  {'max_features': 'auto'}         0.881943        0.291406
1  {'max_features': 'sqrt'}         0.889737        0.281885
2  {'max_features': 'log2'}         0.824431        0.227960


In [30]:
# Lasso

grid(Lasso()).grid_get(X_scaled,y_log,{'alpha': [0.021,0.022,0.023,0.024,0.025,0.026,0.027,0.028,0.029],'max_iter':[10000]})

{'alpha': 0.027, 'max_iter': 10000} 0.8262956310698917
                                params  mean_test_score  std_test_score
0  {'alpha': 0.021, 'max_iter': 10000}         0.826833        0.193505
1  {'alpha': 0.022, 'max_iter': 10000}         0.826634        0.193220
2  {'alpha': 0.023, 'max_iter': 10000}         0.826470        0.192958
3  {'alpha': 0.024, 'max_iter': 10000}         0.826367        0.192685
4  {'alpha': 0.025, 'max_iter': 10000}         0.826307        0.192423
5  {'alpha': 0.026, 'max_iter': 10000}         0.826296        0.192163
6  {'alpha': 0.027, 'max_iter': 10000}         0.826296        0.191908
7  {'alpha': 0.028, 'max_iter': 10000}         0.826314        0.191671
8  {'alpha': 0.029, 'max_iter': 10000}         0.826359        0.191460


In [35]:
# GBR

grid(GradientBoostingRegressor()).grid_get(X_scaled,y_log,{'learning_rate':[0.01,0.011,0.012,0.013,0.014,0.015,0.016,0.017,0.018,0.019,0.02]})

{'learning_rate': 0.01} 0.838083297429105
                      params  mean_test_score  std_test_score
0    {'learning_rate': 0.01}         0.838083        0.277453
1   {'learning_rate': 0.011}         0.838961        0.279584
2   {'learning_rate': 0.012}         0.839815        0.281640
3   {'learning_rate': 0.013}         0.841174        0.282457
4   {'learning_rate': 0.014}         0.841218        0.284069
5   {'learning_rate': 0.015}         0.841465        0.282472
6   {'learning_rate': 0.016}         0.841128        0.279435
7   {'learning_rate': 0.017}         0.840571        0.282089
8   {'learning_rate': 0.018}         0.838831        0.279253
9   {'learning_rate': 0.019}         0.842192        0.276722
10   {'learning_rate': 0.02}         0.842154        0.278049


In [38]:
# SVR

grid(SVR()).grid_get(X_scaled,y_log,{'C':[2.6,2.7,2.8,2.9,3,3.1,3.2,3.3,3.4],'kernel':["linear","poly","rbf"],'degree':[2,3]})

{'C': 2.8, 'degree': 2, 'kernel': 'rbf'} 0.8093798932039262
                                         params  mean_test_score  \
0   {'C': 2.6, 'degree': 2, 'kernel': 'linear'}         0.908344   
1     {'C': 2.6, 'degree': 2, 'kernel': 'poly'}         1.069573   
2      {'C': 2.6, 'degree': 2, 'kernel': 'rbf'}         0.809603   
3   {'C': 2.6, 'degree': 3, 'kernel': 'linear'}         0.908344   
4     {'C': 2.6, 'degree': 3, 'kernel': 'poly'}         0.961477   
5      {'C': 2.6, 'degree': 3, 'kernel': 'rbf'}         0.809603   
6   {'C': 2.7, 'degree': 2, 'kernel': 'linear'}         0.908314   
7     {'C': 2.7, 'degree': 2, 'kernel': 'poly'}         1.076155   
8      {'C': 2.7, 'degree': 2, 'kernel': 'rbf'}         0.809401   
9   {'C': 2.7, 'degree': 3, 'kernel': 'linear'}         0.908314   
10    {'C': 2.7, 'degree': 3, 'kernel': 'poly'}         0.962561   
11     {'C': 2.7, 'degree': 3, 'kernel': 'rbf'}         0.809401   
12  {'C': 2.8, 'degree': 2, 'kernel': 'linear'}         

In [41]:
# Ela

grid(ElasticNet()).grid_get(X_scaled,y_log,{'alpha':[0.04,0.045,0.05,0.055,0.06]})

{'alpha': 0.055} 0.8256211737434089
             params  mean_test_score  std_test_score
0   {'alpha': 0.04}         0.826371        0.194750
1  {'alpha': 0.045}         0.825858        0.194395
2   {'alpha': 0.05}         0.825621        0.194106
3  {'alpha': 0.055}         0.825621        0.193871
4   {'alpha': 0.06}         0.825761        0.193745


In [42]:
# Bay

grid(BayesianRidge()).grid_get(X_scaled,y_log,{'alpha_1':[1,5,10,20]})

{'alpha_1': 10} 0.8173058678934604
            params  mean_test_score  std_test_score
0   {'alpha_1': 1}         0.817367        0.186235
1   {'alpha_1': 5}         0.817331        0.185568
2  {'alpha_1': 10}         0.817306        0.184788
3  {'alpha_1': 20}         0.817310        0.183381


In [43]:
# Extra

grid(ExtraTreesRegressor()).grid_get(X_scaled,y_log,{'max_features':["auto","sqrt","log2"],'bootstrap':[True]})

{'bootstrap': True, 'max_features': 'log2'} 0.8488742670960306
                                        params  mean_test_score  \
0  {'bootstrap': True, 'max_features': 'auto'}         0.854311   
1  {'bootstrap': True, 'max_features': 'sqrt'}         0.887747   
2  {'bootstrap': True, 'max_features': 'log2'}         0.848874   

   std_test_score  
0        0.256476  
1        0.289358  
2        0.256835  


In [46]:
# Ker

grid(KernelRidge()).grid_get(X_scaled,y_log,{'alpha':[1.5,2,2.5,3], 'kernel':["linear","polynomial"], 'degree':[2,3,4],'coef0':[0.1,0.25,0.5,0.75,1]})

{'alpha': 2.5, 'coef0': 0.75, 'degree': 2, 'kernel': 'polynomial'} 0.7994494959495199
                                                params  mean_test_score  \
0    {'alpha': 1.5, 'coef0': 0.1, 'degree': 2, 'ker...         6.578992   
1    {'alpha': 1.5, 'coef0': 0.1, 'degree': 2, 'ker...         0.820066   
2    {'alpha': 1.5, 'coef0': 0.1, 'degree': 3, 'ker...         6.578992   
3    {'alpha': 1.5, 'coef0': 0.1, 'degree': 3, 'ker...         0.863500   
4    {'alpha': 1.5, 'coef0': 0.1, 'degree': 4, 'ker...         6.578992   
..                                                 ...              ...   
115  {'alpha': 3, 'coef0': 1, 'degree': 2, 'kernel'...         0.799597   
116  {'alpha': 3, 'coef0': 1, 'degree': 3, 'kernel'...         4.796994   
117  {'alpha': 3, 'coef0': 1, 'degree': 3, 'kernel'...         0.801481   
118  {'alpha': 3, 'coef0': 1, 'degree': 4, 'kernel'...         4.796994   
119  {'alpha': 3, 'coef0': 1, 'degree': 4, 'kernel'...         0.809314   

     std_test

In [47]:
# Xgb

grid(XGBRegressor()).grid_get(X_scaled,y_log,{'verbosity':[0]})

{'verbosity': 0} 0.8677160531350272
             params  mean_test_score  std_test_score
0  {'verbosity': 0}         0.867716        0.256676


In [55]:
# Ada

grid(AdaBoostRegressor()).grid_get(X_scaled,y_log,{'learning_rate':[0.007,0.008,0.009],'loss':["linear", "square", "exponential"]})

{'learning_rate': 0.008, 'loss': 'exponential'} 0.8545070240846274
                                             params  mean_test_score  \
0        {'learning_rate': 0.002, 'loss': 'linear'}         0.871616   
1        {'learning_rate': 0.002, 'loss': 'square'}         0.867637   
2   {'learning_rate': 0.002, 'loss': 'exponential'}         0.869303   
3        {'learning_rate': 0.003, 'loss': 'linear'}         0.861883   
4        {'learning_rate': 0.003, 'loss': 'square'}         0.858475   
5   {'learning_rate': 0.003, 'loss': 'exponential'}         0.862387   
6        {'learning_rate': 0.004, 'loss': 'linear'}         0.873246   
7        {'learning_rate': 0.004, 'loss': 'square'}         0.874563   
8   {'learning_rate': 0.004, 'loss': 'exponential'}         0.870476   
9        {'learning_rate': 0.005, 'loss': 'linear'}         0.865423   
10       {'learning_rate': 0.005, 'loss': 'square'}         0.871932   
11  {'learning_rate': 0.005, 'loss': 'exponential'}         0.891468 

# Ensemble Methods

In [59]:
class AverageWeight(BaseEstimator, RegressorMixin):
    def __init__(self,mod,weight):
        self.mod = mod
        self.weight = weight
        
    def fit(self,X,y):
        self.models_ = [clone(x) for x in self.mod]
        for model in self.models_:
            model.fit(X,y)
        return self
    
    def predict(self,X):
        w = list()
        pred = np.array([model.predict(X) for model in self.models_])
        # for every data point, single model prediction times weight, then add them together
        for data in range(pred.shape[1]):
            single = [pred[model,data]*weight for model,weight in zip(range(pred.shape[0]),self.weight)]
            w.append(np.sum(single))
        return w

In [56]:
# sorted by performance(good -> bad)
ada = AdaBoostRegressor(learning_rate = 0.715, loss = 'square')
bay = BayesianRidge()
svr = SVR(C = 2.8, degree = 2, kernel = 'rbf')
ela = ElasticNet(alpha = 0.055)
ridge = Ridge(alpha = 84)
rf = RandomForestRegressor(max_features = 'log2')
extra = ExtraTreesRegressor(bootstrap = True, max_features = 'log2')
xgb = XGBRegressor(verbosity = 0)


In [57]:
# assign weights based on their gridsearch score

w1 = 0.2
w2 = 0.15
w3 = 0.15
w4 = 0.15
w5 = 0.15
w6 = 0.1
w7 = 0.05
w8 = 0.05

In [60]:
weight_avg = AverageWeight(mod = [ada,bay,svr,ela,ridge,rf,extra,xgb],weight=[w1,w2,w3,w4,w5,w6,w7,w8])

In [61]:
rmse_cv(weight_avg,X_scaled,y_log),  rmse_cv(weight_avg,X_scaled,y_log).mean()

(array([0.69620746, 0.6716779 , 0.91735013, 1.00526252, 0.66503932]),
 0.7900176416641335)

# Stacking

In [62]:
class stacking(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self,mod,meta_model):
        self.mod = mod
        self.meta_model = meta_model
        self.kf = KFold(n_splits=5, random_state=42, shuffle=True)
        
    def fit(self,X,y):
        self.saved_model = [list() for i in self.mod]
        oof_train = np.zeros((X.shape[0], len(self.mod)))
        
        for i,model in enumerate(self.mod):
            for train_index, val_index in self.kf.split(X,y):
                renew_model = clone(model)
                renew_model.fit(X[train_index], y[train_index])
                self.saved_model[i].append(renew_model)
                oof_train[val_index,i] = renew_model.predict(X[val_index])
        
        self.meta_model.fit(oof_train,y)
        return self
    
    def predict(self,X):
        whole_test = np.column_stack([np.column_stack(model.predict(X) for model in single_model).mean(axis=1) 
                                      for single_model in self.saved_model]) 
        return self.meta_model.predict(whole_test)
    
    def get_oof(self,X,y,test_X):
        oof = np.zeros((X.shape[0],len(self.mod)))
        test_single = np.zeros((test_X.shape[0],5))
        test_mean = np.zeros((test_X.shape[0],len(self.mod)))
        for i,model in enumerate(self.mod):
            for j, (train_index,val_index) in enumerate(self.kf.split(X,y)):
                clone_model = clone(model)
                clone_model.fit(X[train_index],y[train_index])
                oof[val_index,i] = clone_model.predict(X[val_index])
                test_single[:,j] = clone_model.predict(test_X)
            test_mean[:,i] = test_single.mean(axis=1)
        return oof, test_mean

In [63]:
# must do imputer first, otherwise stacking won't work, and i don't know why.
a = Imputer().fit_transform(X_scaled)
b = Imputer().fit_transform(y_log.values.reshape(-1,1)).ravel()

C:\Users\Octavia\.conda\envs\MSBD5001\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Octavia\.conda\envs\MSBD5001\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [64]:
stack_model = stacking(mod = [ada,bay,svr,ela,ridge,rf,extra,xgb], meta_model = ada)

In [65]:
print(rmse_cv(stack_model,a,b))
print(rmse_cv(stack_model,a,b).mean())

[0.78517415 0.79789367 0.91091349 1.04361989 0.86958781]
0.8774763008352467


### Extract the features generated from stacking, and combine them with original features.

In [66]:
X_train_stack, X_test_stack = stack_model.get_oof(a,b,test_X_scaled)

In [67]:
X_train_stack.shape, a.shape

((340, 8), (340, 26))

In [68]:
X_train_add = np.hstack((a,X_train_stack))

In [69]:
X_test_add = np.hstack((test_X_scaled,X_test_stack))

In [70]:
X_train_add.shape, X_test_add.shape

((340, 34), (90, 34))

In [71]:
print(rmse_cv(stack_model,X_train_add,b))
print(rmse_cv(stack_model,X_train_add,b).mean())

[0.77663277 0.7871371  0.91806911 1.0845662  0.85772566]
0.8901704755304781


# Submission

In [72]:
# Final Model

stack_model = stacking(mod = [ada,bay,svr,ela,ridge,rf,extra,xgb], meta_model = ada)

In [73]:
stack_model.fit(a,b)

stacking(meta_model=AdaBoostRegressor(base_estimator=None, learning_rate=0.715,
                                      loss='square', n_estimators=50,
                                      random_state=None),
         mod=[AdaBoostRegressor(base_estimator=None, learning_rate=0.715,
                                loss='square', n_estimators=50,
                                random_state=None),
              BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False,
                            copy_X=True, fit_intercept=True, lambda_1=1e-06,
                            lam...
              XGBRegressor(base_score=0.5, booster='gbtree',
                           colsample_bylevel=1, colsample_bynode=1,
                           colsample_bytree=1, gamma=0, importance_type='gain',
                           learning_rate=0.1, max_delta_step=0, max_depth=3,
                           min_child_weight=1, missing=None, n_estimators=100,
                           n_jobs=1, nthread=N

In [74]:
pred = np.exp(stack_model.predict(test_X_scaled) + 1)

In [75]:
result=pd.DataFrame({'id':range(len(df_test2)), 'playtime_forever':pred})


In [76]:
result.to_csv("./Submission/submission_1122_PCA_FineTune(1).csv",index=False)